# Libraries

In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import json
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.svm import SVC

# Loading the processed Data

In [2]:
X_train = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/X_train.csv')
X_test = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/X_test.csv')
y_train = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/y_train.csv')['Class']
y_test = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/y_test.csv')['Class']
X_train_res = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/X_train_res.csv')
y_train_res = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/y_train_res.csv')['Class']

In [3]:
y_train = y_train.ravel()
y_test = y_test.ravel()
y_train_res = y_train_res.ravel()

# Model

## Training on imbalanced data

In [4]:
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.001, 0.01],
    'class_weight': [{0: 1, 1: w} for w in [10, 50, 100]]
}

In [6]:
grid_search = GridSearchCV(
    SVC(kernel='rbf', probability=True, random_state=42),
    param_grid,
    cv=3,
    scoring='precision',
    n_jobs=-1
)

In [8]:
grid_search.fit(X_train, y_train)

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/joblib/__init__.py", line 114, in <module>
    from .memory import Memory
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/joblib/memory.py", line 12, in <module>
    import asyncio
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/__init__.py", line 8, in <module>
    from .base_events import *
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/base_events.py", line 39, in <module>
    from . import coroutines
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/coroutines.py", line 4, in <module>
    import inspect
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/inspect.py", line 146, in <module

KeyboardInterrupt: 

In [ ]:
print("Best parameters:", grid_search.best_params_)
best_svm = grid_search.best_estimator_

In [ ]:
y_pred = best_svm.predict(X_test)